In [2]:
from tweepy.streaming import StreamListener

In [1]:
%%time

import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql import functions as f
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql import DataFrame
import re

spark = SparkSession \
        .builder \
        .appName("ML Model Building") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .config("spark.driver.memory","4g") \
        .config("spark.executor.memory","5g") \
        .master("local") \
        .getOrCreate()


CPU times: total: 109 ms
Wall time: 5.25 s


In [29]:


import tweepy
from textblob import TextBlob
from kafka import KafkaProducer
from json import dumps


with open('config.json') as json_file:
   data = json.load(json_file)

twitter_config = config["twitter"]
CONSUMER_KEY = twitter_config['consumer_key']
CONSUMER_SECRET = twitter_config['consumer_secret']
ACCESS_TOKEN = twitter_config['access_token']
ACCESS_TOKEN_SECRET = twitter_config['access_secret']


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda K:dumps(K).encode('utf-8'))


api = tweepy.API(auth)
cursor = tweepy.Cursor(api.search,q="USA",tweet_mode='extended').items(10)
for tweet in cursor:
   producer.send('twitter_analysis..',tweet.full_text)

TweepError: Twitter error response: status code = 403